In [ ]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output

## Nifty 50 OI

In [ ]:
nifty_50 = pd.read_csv('../data/ind_nifty50list.csv')
inst = pd.DataFrame(kite.instruments("NFO"))
inst.rename(columns={'name':'Symbol'},inplace=True)
inst_cols = ['Symbol','instrument_token','tradingsymbol','last_price','lot_size','strike','instrument_type']
inst = inst[inst_cols]
inst = nifty_50.merge(inst, how = 'inner', on  = 'Symbol')

In [ ]:
# inst.tradingsymbol.unique().tolist()
start_dt = '2023-03-21'
end_dt = '2023-03-21'
time_frame = 'minute'
bn_token  = 260105

for i in inst.Symbol.unique()[11:]:
    try:
        print(i)
        df_inst_expiry = inst[(inst['Symbol'] == i) & (inst.tradingsymbol.str.contains('23MAR')) & ~(inst.tradingsymbol.str.contains('FUT'))]
        df_inst_expiry['strike_type'] = df_inst_expiry['strike'].astype(int).astype(str) + df_inst_expiry['instrument_type']

        df_hist_data = pd.DataFrame()
        inst_expiry = df_inst_expiry['instrument_token'].unique().tolist()
        strike_symbol_dict = dict(zip(df_inst_expiry.instrument_token,df_inst_expiry['strike_type'] ))
        for j in inst_expiry:
            df = pd.DataFrame(kite.historical_data(j, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
            df['strike_type'] = strike_symbol_dict[j]
            df_hist_data = df_hist_data.append(df)
        df_hist_data['oi_change'] = df_hist_data.groupby(['strike_type']).oi.diff()
        df_hist_data['oi_change'] = df_hist_data.groupby(['strike_type']).oi_change.cumsum()
        df_hist_data_1 = df_hist_data.groupby('date').agg(
        {
        "oi_change": lambda x: sorted(x,reverse=True)[0:5]
        }
        ).reset_index().explode('oi_change')
        df_hist_data_1 = df_hist_data_1.merge(
        df_hist_data[['date','oi_change','strike_type']], how = 'left',on = ['date','oi_change']
        ).dropna()
        df_hist_data_1['call_put'] = df_hist_data_1.strike_type.str[-2:]
        df_hist_data_1['Symbol'] = i
        df_hist_data_1.to_csv(f'data/{i}.csv',index = False)
    except:
        print("fail", i)